In [5]:
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker.debugger import Rule
from sagemaker.debugger import rule_configs


from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker



In [12]:
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

sm_sess = sagemaker.session.Session()

AWS Region: eu-west-1
RoleArn: arn:aws:iam::467432373215:role/service-role/AmazonSageMaker-ExecutionRole-20221206T164397


In [13]:
bucket = sagemaker.Session().default_bucket()
bucket

'sagemaker-eu-west-1-467432373215'

In [14]:
prefix = "demo-sagemaker-boston"

In [15]:
s3_output_location='s3://{}/{}/{}'.format(bucket, prefix, 'knn_model')
s3_output_location

's3://sagemaker-eu-west-1-467432373215/demo-sagemaker-boston/knn_model'

## Entrenamiento del modelo k-nn
- https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/k-nearest-neighbors.html

In [16]:
container=sagemaker.image_uris.retrieve("knn", region)
print(container)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


438346466558.dkr.ecr.eu-west-1.amazonaws.com/knn:1


---

### Formato de entrada
- https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/kNN-in-formats.html

In [17]:
train_input = TrainingInput(
    f"s3://{bucket}/{prefix}/data/train.csv", 
    content_type="text/csv",
)
validation_input = TrainingInput(
    f"s3://{bucket}/{prefix}/data/validation.csv",
    content_type="text/csv",
)

In [7]:
import time
from time import strftime

In [8]:
create_date = strftime("%Y-%m-%d-%H-%M-%S")
demo_experiment = Experiment.create(experiment_name = "DEMO-{}".format(create_date),
                                    description = "Demo experiment",
                                    tags = [{'Key': 'demo-experiments', 'Value': 'demo1'}])

In [9]:
demo_trial = Trial.create(trial_name = "DEMO-{}".format(create_date),
                          experiment_name = demo_experiment.experiment_name,
                          tags = [{'Key': 'demo-trials', 'Value': 'demo1'}])

In [18]:
knn = sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=s3_output_location,
    sagemaker_session=sm_sess,
    tags = [{'Key': 'demo-jobs', 'Value': 'demo2'}]
)

Hyperparametros: https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/kNN_hyperparameters.html

In [19]:
knn.set_hyperparameters(
    feature_dim= 13,
    k = 3, 
    sample_size = 303, 
    predictor_type = 'regressor'
)

___

In [20]:
knn.fit({"train": train_input, "validation": validation_input}, wait=True)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: knn-2022-12-07-11-14-06-051


2022-12-07 11:14:06 Starting - Starting the training job...
2022-12-07 11:14:29 Starting - Preparing the instances for trainingProfilerReport-1670411646: InProgress
......
2022-12-07 11:15:31 Downloading - Downloading input data
2022-12-07 11:15:31 Training - Downloading the training image..................
2022-12-07 11:18:32 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[12/07/2022 11:18:34 INFO 139834377893696] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-conf.json: {'_kvstore': 'dist_async', '_log_level': 'info', '_num_gpus': 'auto', '_num_kv_servers': '1', '_tuning_objective_metric': '', '_faiss_index_nprobe': '5', 'epochs': '1', 'feature_dim': 'auto', 'faiss_index_ivf_nlists': 'auto', 'index_metric': 'L2', 'index_type': 'faiss.Flat', 'mini_batch_size': '5000', '_enable_profiler': 'false'}
[12/07/2022 11:1

In [21]:
knn.output_path

's3://sagemaker-eu-west-1-467432373215/demo-sagemaker-boston/knn_model'

In [22]:
knn.latest_training_job.name

'knn-2022-12-07-11-14-06-051'

In [23]:
rule_output_path = knn.output_path + "/" + knn.latest_training_job.name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive

2022-12-07 11:19:01     329718 demo-sagemaker-boston/knn_model/knn-2022-12-07-11-14-06-051/rule-output/ProfilerReport-1670411646/profiler-output/profiler-report.html
2022-12-07 11:19:00     171104 demo-sagemaker-boston/knn_model/knn-2022-12-07-11-14-06-051/rule-output/ProfilerReport-1670411646/profiler-output/profiler-report.ipynb
2022-12-07 11:18:57        191 demo-sagemaker-boston/knn_model/knn-2022-12-07-11-14-06-051/rule-output/ProfilerReport-1670411646/profiler-output/profiler-reports/BatchSize.json
2022-12-07 11:18:57        199 demo-sagemaker-boston/knn_model/knn-2022-12-07-11-14-06-051/rule-output/ProfilerReport-1670411646/profiler-output/profiler-reports/CPUBottleneck.json
2022-12-07 11:18:57        126 demo-sagemaker-boston/knn_model/knn-2022-12-07-11-14-06-051/rule-output/ProfilerReport-1670411646/profiler-output/profiler-reports/Dataloader.json
2022-12-07 11:18:57        127 demo-sagemaker-boston/knn_model/knn-2022-12-07-11-14-06-051/rule-output/ProfilerReport-1670411646/pr

In [15]:
knn.model_data

's3://sagemaker-eu-west-1-467432373215/demo-sagemaker-boston/knn_model/knn-2022-12-07-10-53-12-387/output/model.tar.gz'

___
# Deploy

In [16]:
from sagemaker.predictor import csv_serializer, json_deserializer

In [ ]:
knn_predictor=knn.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    #serializer=CSVSerializer()
)
knn_predictor.serializer = csv_serializer
knn_predictor.deserializer = json_deserializer

-

## Test result

In [81]:
import pandas as pd
import numpy as np

In [102]:
data = pd.read_csv(f"s3://{bucket}/{prefix}/data/validation.csv", header=None)

In [105]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,8.8,20.08490,0.0,18.10,0.0,0.700,4.368,91.2,1.4395,24.0,666.0,20.2,285.83,30.63
1,20.8,12.04820,0.0,18.10,0.0,0.614,5.648,87.6,1.9512,24.0,666.0,20.2,291.55,14.10
2,17.4,0.13554,12.5,6.07,0.0,0.409,5.594,36.8,6.4980,4.0,345.0,18.9,396.90,13.09
3,11.9,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88
4,29.8,0.12579,45.0,3.44,0.0,0.437,6.556,29.1,4.5667,5.0,398.0,15.2,382.84,4.56


In [103]:
y = data.iloc[:, 0].values
x = data.iloc[:, 1:].values

In [104]:
print(y.shape, x.shape)

(101,) (101, 13)


In [107]:
result = knn_predictor.predict(x)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_deserializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [108]:
# Convert JSON predictions to an array
preds_array = np.array([result['predictions'][i]['predicted_label'] for i in range(len(result['predictions']))])

In [119]:
np.power(y - preds_array, 2).mean()

35.27848193687974

### Delete deployed model

In [ ]:
boto3.client("sagemaker").delete_endpoint(EndpointName=predictor.endpoint_name)